<a href="https://colab.research.google.com/github/maximealive/GoogleColab_save_weights_epochs/blob/master/GoogleColab_save_weights_epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import os, subprocess
import pathlib
import numpy as np
import time
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.optimizers as optimizers
from keras.callbacks import *

from __future__ import absolute_import, division, print_function, unicode_literals

# Mount Google Drive and create folders
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

work_dir = '/content/work_dir'                      # local work directory (faster)               
dwld_dir = '/content/gdrive/My Drive/dwld_dir'      # drive download directory 
subprocess.call(["mkdir","-p",work_dir])            # create directory for work
subprocess.call(["mkdir","-p",dwld_dir])            # create directory for dataset download

os.chdir(dwld_dir)
if not os.path.exists(dwld_dir + '/GoogleColab_save_weights_epochs'):
  # Download from github repository
  !git clone -l -s https://github.com/maximealive/GoogleColab_save_weights_epochs.git

# Copy dataset(.zip) from 'dwld_dir/GoogleColab_save_weights_epochs' to 'work_dir' directory
!cp /content/gdrive/My\ Drive/dwld_dir/GoogleColab_save_weights_epochs/flower_photos.zip /content/work_dir/

# Unzip dataset if not available
os.chdir(work_dir)            
if os.path.isdir('./flower_photos') == False: 
  !unzip -o -q flower_photos.zip -d .

train_dir = pathlib.Path(work_dir + '/flower_photos/train/')
test_dir = pathlib.Path(work_dir + '/flower_photos/test/')

# Get class labels and number of classes
CLASS_NAMES = np.array([item.name for item in train_dir.glob('*')])         # classes are subdirectories of '/ train /'
NUM_CLASSES = len(CLASS_NAMES)                                              # total number of classes

img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

BATCH_SIZE = 8        # number of elements processed together in each iteration
IMG_HEIGHT = 32
IMG_WIDTH = 32
IMG_CHANNELS = 3
EPOCHS = 10           # number of training times

# Create ImageDataGenerator by converting from uint8 to float32 in the range [0,1]
train_data_gen = img_gen.flow_from_directory(directory=str(train_dir),
                                          target_size=(IMG_HEIGHT, IMG_WIDTH),
                                          color_mode='rgb',                                                     
                                          classes = list(CLASS_NAMES),
                                          class_mode='categorical',
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

test_data_gen = img_gen.flow_from_directory(directory=str(test_dir),
                                          target_size=(IMG_HEIGHT, IMG_WIDTH),
                                          color_mode='rgb',                                                     
                                          classes = list(CLASS_NAMES),
                                          class_mode='categorical',
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

# Define a model
def create_model():

  model = models.Sequential()
  model.add(layers.Conv2D(filters=6, 
                          kernel_size=(3, 3), 
                          strides = (1,1),
                          padding = 'valid',
                          activation='relu', 
                          input_shape=(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)))
  model.add(layers.AveragePooling2D())
  model.add(layers.Conv2D(filters=16, 
                          kernel_size=(3, 3),
                          strides = (1,1),
                          padding = 'valid', 
                          activation='relu'))
  model.add(layers.AveragePooling2D())
  model.add(layers.Flatten())
  model.add(layers.Dense(units=120, activation='relu'))
  model.add(layers.Dense(units=84, activation='relu'))
  model.add(layers.Dense(units=NUM_CLASSES, activation = 'softmax'))

  LEARNING_RATE = 0.001 
  model.compile(optimizer=optimizers.Adam(lr=LEARNING_RATE),            
              loss='categorical_crossentropy',            
              metrics=['accuracy'])
  return model
  
# Create a basic model instance
model = create_model()

checkpoint_path = '/content/gdrive/My Drive/_Projects/Git_Hub/weights/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# If the directory contains checkpoints they will be loaded
if os.listdir(checkpoint_dir):
  print('\n###################################################\n')
  print('\n             Restoring Weights & Epoch             \n')
  print('\n###################################################\n')

  # Obtain latest checkpoint
  latest = tf.train.latest_checkpoint(checkpoint_dir)
  
  # Number of epochs remaining
  EPOCH_UPGRADED = int(latest[55:-5])
  EPOCH_UPGRADED = EPOCHS - EPOCH_UPGRADED

  # Load the previously saved weights and train the model
  model.load_weights(latest)

  # run on GPU
  with tf.device('/GPU:0'):
    history = model.fit_generator(
        generator=train_data_gen,
        steps_per_epoch=train_data_gen.n // BATCH_SIZE,      
        validation_data=test_data_gen,
        validation_steps=test_data_gen.n // BATCH_SIZE,
        epochs=EPOCH_UPGRADED,
        verbose=1,
        callbacks=[cp_callback], 
        use_multiprocessing = True
        )
else:
  # run on GPU
  # Train the model
  with tf.device('/GPU:0'):
    history = model.fit_generator(
        generator=train_data_gen,
        steps_per_epoch=train_data_gen.n // BATCH_SIZE,      
        validation_data=test_data_gen,
        validation_steps=test_data_gen.n // BATCH_SIZE,
        epochs=EPOCHS,
        verbose=1,
        callbacks=[cp_callback], 
        use_multiprocessing = True
        )

# run on GPU   
# Evaluate the model
with tf.device('/GPU:0'):
  test_loss, test_acc = model.evaluate_generator(generator=test_data_gen, 
                                                 steps = test_data_gen.n // BATCH_SIZE, 
                                                 verbose=1)

print(f"Model accuracy: {100*test_acc:.2f}%")